In [1]:
import os
os.chdir("..")

In [2]:
import pandas as pd
from pathlib import Path

In [17]:
import warnings
warnings.filterwarnings("ignore")

In [26]:
# Data is huge, ~15M records per month. Use only necessary columns.
# Load last three month data and make ready a pandas dataframe.

def load_month(path):
    return pd.read_parquet(path, columns=REQ_COLUMNS)

REQ_COLUMNS = [
    "station_name",
    # "final_destination_station",
    "train_name",
    "delay_in_min",
    "time",
    "is_canceled"
]

data_dir = Path("data")

files = [
    data_dir / "data-2025-10.parquet",
    data_dir / "data-2025-11.parquet",
    data_dir / "data-2025-12.parquet",
]

df = pd.concat([load_month(f) for f in files], ignore_index=True)

In [27]:
# Basic cleaning
# Remove cancelled trains, to focus delay magnitude
# !NOTE: Should we reatin cancelled ones?

df = df[
    (df["is_canceled"] == False) &
    (df["delay_in_min"].notna())
]

df["time"] = pd.to_datetime(df["time"])
df["date"] = df["time"].dt.date

In [28]:
# Create a new column to uniquely identify the route

df["route_id"] = (
    df["train_name"].astype(str) + " → " + df["station_name"].astype(str)
)

In [30]:
# Sample data: only include top and busiest routes

route_counts = (
    df.groupby("route_id")
      .size()
      .sort_values(ascending=False)
)

# Keep top N routes
TOP_N_ROUTES = 150

major_routes = route_counts.head(TOP_N_ROUTES).index

df = df[df["route_id"].isin(major_routes)]

In [31]:
df

,station_name,train_name,delay_in_min,time,is_canceled,date,route_id
1,München Ost,S 1,7,2025-10-01 00:00:00,False,2025-10-01,S 1 → München Ost
6,Düsseldorf Hbf,S 6,0,2025-10-01 00:00:00,False,2025-10-01,S 6 → Düsseldorf Hbf
13,Hamburg Hbf,S 3,4,2025-10-01 00:00:00,False,2025-10-01,S 3 → Hamburg Hbf
15,Berlin Potsdamer Platz,S 1,1,2025-10-01 00:00:00,False,2025-10-01,S 1 → Berlin Potsdamer Platz
16,Frankfurt (Main) Hbf,S 6,1,2025-10-01 00:00:00,False,2025-10-01,S 6 → Frankfurt (Main) Hbf
...,...,...,...,...,...,...,...
31389297,Bremen Hbf,NWB RS1,0,2025-12-31 23:34:00,False,2025-12-31,NWB RS1 → Bremen Hbf
31390833,Brebach,STB 1,0,2025-12-31 23:39:00,False,2025-12-31,STB 1 → Brebach
31393022,Brebach,STB 1,0,2025-12-31 23:47:00,False,2025-12-31,STB 1 → Brebach
31394972,Brebach,STB 1,0,2025-12-31 23:54:00,False,2025-12-31,STB 1 → Brebach


In [34]:
daily_route_delays = (
    df.groupby(["route_id", "date"])
      .agg(
          mean_delay=("delay_in_min", "mean"),
          sd_delay=("delay_in_min", "std"),
          n_trains=("delay_in_min", "size")
      )
      .reset_index()
)


daily_route_delays["date"] = pd.to_datetime(daily_route_delays["date"])
daily_route_delays["weekday"] = daily_route_delays["date"].dt.weekday
daily_route_delays["is_weekend"] = daily_route_delays["weekday"] >= 5

In [35]:
daily_route_delays

,route_id,date,mean_delay,sd_delay,n_trains,weekday,is_weekend
0,NWB RS1 → Bremen Hbf,2025-10-01,4.034483,4.383628,116,2,False
1,NWB RS1 → Bremen Hbf,2025-10-02,2.546875,3.111319,64,3,False
2,NWB RS1 → Bremen Hbf,2025-10-03,1.457143,1.578213,35,4,False
3,NWB RS1 → Bremen Hbf,2025-10-04,1.818182,2.805913,44,5,True
4,NWB RS1 → Bremen Hbf,2025-10-05,1.866667,2.431593,30,6,True
...,...,...,...,...,...,...,...
8323,STB 1 → Brebach,2025-12-27,0.000000,0.000000,226,5,True
8324,STB 1 → Brebach,2025-12-28,0.000000,0.000000,147,6,True
8325,STB 1 → Brebach,2025-12-29,0.000000,0.000000,258,0,False
8326,STB 1 → Brebach,2025-12-30,0.000000,0.000000,258,1,False
